In [ ]:
import json, logging
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import make_scorer
from pathlib import Path
from datetime import datetime
from scipy.stats import skew, kurtosis

In [ ]:
NOTEBOOK_NAME = "MIPLIB-XGBoost-Default"

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
RUN_DIR = Path("../results") / f"{timestamp}_{NOTEBOOK_NAME}"
SUBDIRS = ["model", "optuna", "metrics", "logs"]

for sd in SUBDIRS:
    (RUN_DIR / sd).mkdir(parents=True, exist_ok=True)

print(f"All outputs will be saved under: {RUN_DIR.resolve()}")

def save_model(model, name="xgb_runtime_model"):
    model_path = RUN_DIR / "model" / f"{name}.json"
    model.save_model(model_path)
    print(f"Model saved to {model_path}")


def save_optuna(study):
    joblib.dump(study, RUN_DIR / "optuna" / "study.pkl")
    trials_df = study.trials_dataframe()
    trials_df.to_csv(RUN_DIR / "optuna" / "trials.csv", index=False)
    with open(RUN_DIR / "optuna" / "best_params.json", "w") as f:
        json.dump(study.best_params, f, indent=2)
    print("Optuna study & best params stored")


def save_metrics(name: str, value):
    with open(RUN_DIR / "metrics" / f"{name}.json", "w") as f:
        json.dump(value, f, indent=2)

logfile = RUN_DIR / "logs" / "run.log"
logging.basicConfig(filename=logfile, filemode="w",
                    level=logging.INFO,
                    format="%(asctime)s %(levelname)s | %(message)s")
logging.info("Run directory initialised")

In [15]:
df_features = pd.read_csv('data/miplib_hutter_features.csv', header=0)
df_cplex_runtimes = pd.read_csv('/data/miplib_cplex_runtimes.csv')
df_cplex_runtimes["runtime_sec"] = pd.to_numeric(df_cplex_runtimes["runtime_sec"])
df_features.head()

,instance,probtype,n_vars,n_constr,n_nzcnt,nq_vars,nq_constr,nq_nzcnt,lp_avg,lp_l2_avg,...,cliqueCuts,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time
0,30n20b8.mps.gz,1,18380.0,576,109706.0,0,0,0,0.451208,8.062840,...,0,0,0,0,0,0.47,0.01,0.00,0.01,4.21
1,2club200v15p5scn.mps.gz,1,200.0,17013,104811.0,0,0,0,0.310556,0.362519,...,0,0,0,0,0,0.30,0.01,0.00,0.02,9.10
2,8div-n59k10.mps.gz,1,6143.0,2065,539151.0,0,0,0,0.242108,9.867550,...,0,0,0,0,0,0.52,0.03,0.01,0.03,4.65
3,22433.mps.gz,1,429.0,198,3408.0,0,0,0,0.117360,0.200469,...,0,0,0,0,0,0.00,0.00,0.00,0.00,10.59
4,10teams.mps.gz,1,2025.0,230,12150.0,0,0,0,0.020263,0.081960,...,0,0,0,0,0,0.02,0.00,0.00,0.00,16.77


In [16]:
df_cplex_runtimes.head()

,instance,runtime_sec
0,a2864-99blp.mps.gz,3600.0
1,adult-regularized.mps.gz,3600.0
2,allcolor58.mps.gz,3600.0
3,assign1-10-4.mps.gz,3600.0
4,bab3.mps.gz,3600.0


In [17]:
df_combined = (
    df_features                         # left table
      .merge(                           # join
          df_cplex_runtimes[["instance", "runtime_sec"]],  # right table (only needed cols)
          on="instance",                # key column
          how="inner"                   # inner = intersection
      )
)
df_combined.head()

,instance,probtype,n_vars,n_constr,n_nzcnt,nq_vars,nq_constr,nq_nzcnt,lp_avg,lp_l2_avg,...,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time,runtime_sec
0,30n20b8.mps.gz,1,18380.0,576,109706.0,0,0,0,0.451208,8.062840,...,0,0,0,0,0.47,0.01,0.00,0.01,4.21,2.13
1,2club200v15p5scn.mps.gz,1,200.0,17013,104811.0,0,0,0,0.310556,0.362519,...,0,0,0,0,0.30,0.01,0.00,0.02,9.10,3608.04
2,8div-n59k10.mps.gz,1,6143.0,2065,539151.0,0,0,0,0.242108,9.867550,...,0,0,0,0,0.52,0.03,0.01,0.03,4.65,3600.22
3,22433.mps.gz,1,429.0,198,3408.0,0,0,0,0.117360,0.200469,...,0,0,0,0,0.00,0.00,0.00,0.00,10.59,0.24
4,10teams.mps.gz,1,2025.0,230,12150.0,0,0,0,0.020263,0.081960,...,0,0,0,0,0.02,0.00,0.00,0.00,16.77,2.92


In [18]:
print(df_combined.shape)

(922, 150)


In [19]:
# --- from the df_combined you just built -----------------
# 1)   X   → all predictors (no target)
df_features = df_combined.drop(columns=["runtime_sec"])

# 2)   y   → just the target column
df_cplex_runtimes = df_combined[["runtime_sec"]].copy()

# (Optional but common) make the instance name the index so the two line up row‑for‑row:
df_features.set_index("instance", inplace=True)
df_cplex_runtimes.index = df_features.index

In [20]:
df_features.head()

,probtype,n_vars,n_constr,n_nzcnt,nq_vars,nq_constr,nq_nzcnt,lp_avg,lp_l2_avg,lp_linf,...,cliqueCuts,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time
instance,,,,,,,,,,,,,,,,,,,,,
30n20b8.mps.gz,1,18380.0,576,109706.0,0,0,0,0.451208,8.062840,188.846000,...,0,0,0,0,0,0.47,0.01,0.00,0.01,4.21
2club200v15p5scn.mps.gz,1,200.0,17013,104811.0,0,0,0,0.310556,0.362519,0.500000,...,0,0,0,0,0,0.30,0.01,0.00,0.02,9.10
8div-n59k10.mps.gz,1,6143.0,2065,539151.0,0,0,0,0.242108,9.867550,708.000000,...,0,0,0,0,0,0.52,0.03,0.01,0.03,4.65
22433.mps.gz,1,429.0,198,3408.0,0,0,0,0.117360,0.200469,0.466942,...,0,0,0,0,0,0.00,0.00,0.00,0.00,10.59
10teams.mps.gz,1,2025.0,230,12150.0,0,0,0,0.020263,0.081960,0.500000,...,0,0,0,0,0,0.02,0.00,0.00,0.00,16.77


In [21]:
df_cplex_runtimes.head()

,runtime_sec
instance,
30n20b8.mps.gz,2.13
2club200v15p5scn.mps.gz,3608.04
8div-n59k10.mps.gz,3600.22
22433.mps.gz,0.24
10teams.mps.gz,2.92


In [23]:
# Steps from Hutter et al.(compare preeliminaries):
# 1. Remove constant columns
# 2. Replace the sentinel –512 with NaN
# 3. Standardise every remaining column to mean 0 / std 1. The means/SDs are computed ignoring NaNs.
# 4. Fill the remaining NaNs with 0, so a “missing” value is interpreted as “the mean of that column” after scaling.


# instance id does nothing for performance prediction
#df_features.drop(['instance'], inplace=True, axis=1)
#df_cplex_runtimes.drop(['instance'], inplace=True, axis=1)
# Some rows have the same values - we won't be needing them as they only increase complexity.
single_value_cols = df_features.columns[df_features.nunique(dropna=False) == 1]
single_value_cols

Index(['probtype', ' nq_vars', ' nq_constr', ' nq_nzcnt', ' num_s_variables',
       ' num_n_variables', ' ratio_s_variables', ' ratio_n_variables',
       ' itcnt_max', ' numnewsolution_sum', ' newin_sum', ' nodeleft_avg',
       ' nodeleft_varcoef', ' diffObj_avg', ' diffObj_median',
       ' diffObj_varcoef', ' diffObj_q90mq10', ' numfeas', ' iinf_avg',
       ' iinf_median', ' iinf_varcoef', ' iinf_q90mq10', ' diffBestInt_avg',
       ' diffBestInt_median', ' diffBestInt_varcoef', ' diffBestInt_q90mq10',
       ' diffBestObjUp_avg', ' diffBestObjUp_median', ' diffBestObjUp_varcoef',
       ' diffBestObjUp_q90mq10', ' numcuts_sum', ' diffGap_avg',
       ' diffGap_median', ' diffGap_varcoef', ' diffGap_q90mq10', ' pre_t',
       ' rel_t', ' new_row', ' new_col', ' new_nonzero', ' clique_table',
       ' cliqueCuts', ' impliedBoundCuts', ' flowCuts',
       ' mixedIntegerRoundingCuts', ' gomoryFractionalCuts'],
      dtype='object')

In [24]:
df_features.drop(single_value_cols, axis=1, inplace=True)
df_features

,n_vars,n_constr,n_nzcnt,lp_avg,lp_l2_avg,lp_linf,lp_objval,num_b_variables,num_i_variables,num_c_variables,...,obj_coef_per_sqr_constr2_std,mipgap,nodecnt,clqcnt,covcnt,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time
instance,,,,,,,,,,,,,,,,,,,,,
30n20b8.mps.gz,18380.0,576,109706.0,0.451208,8.062840,188.846000,0.390104,18318.0,62,0,...,0.056731,1.000000,0,0,0,0.47,0.01,0.00,0.01,4.21
2club200v15p5scn.mps.gz,200.0,17013,104811.0,0.310556,0.362519,0.500000,-121.222000,200.0,0,0,...,0.004690,0.765000,0,0,0,0.30,0.01,0.00,0.02,9.10
8div-n59k10.mps.gz,6143.0,2065,539151.0,0.242108,9.867550,708.000000,-709.000000,6138.0,5,0,...,0.006379,1.000000,0,0,0,0.52,0.03,0.01,0.03,4.65
22433.mps.gz,429.0,198,3408.0,0.117360,0.200469,0.466942,21240.500000,231.0,0,198,...,0.048224,0.000000,0,0,0,0.00,0.00,0.00,0.00,10.59
10teams.mps.gz,2025.0,230,12150.0,0.020263,0.081960,0.500000,917.000000,1800.0,0,225,...,6.408910,0.086580,0,5,0,0.02,0.00,0.00,0.00,16.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z26.mps.gz,17937.0,850513,1715610.0,0.066037,0.165127,0.500000,-1651.940000,17937.0,0,0,...,0.114323,0.998272,0,0,0,4.49,0.62,0.09,0.65,7.73
8div-n59k12.mps.gz,24575.0,8211,8448020.0,0.251794,19.749700,2844.000000,-2845.000000,24570.0,5,0,...,0.003189,1.000000,0,0,0,37.19,0.42,0.04,0.42,5.89
bley_xl1.mps.gz,5831.0,175620,869391.0,0.056589,0.142278,0.500000,154.390000,5831.0,0,0,...,18.397400,0.000000,0,121,8,154.16,0.15,0.02,0.15,11.55


In [25]:
df_features.dtypes

 n_vars             float64
 n_constr             int64
 n_nzcnt            float64
 lp_avg             float64
 lp_l2_avg          float64
                     ...   
time_relax          float64
 time_VCG0          float64
 time_VCG1          float64
 time_VCG2          float64
 cplex_prob_time    float64
Length: 102, dtype: object

In [26]:
df_features.replace(-512, np.nan, inplace=True)
df_features

,n_vars,n_constr,n_nzcnt,lp_avg,lp_l2_avg,lp_linf,lp_objval,num_b_variables,num_i_variables,num_c_variables,...,obj_coef_per_sqr_constr2_std,mipgap,nodecnt,clqcnt,covcnt,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time
instance,,,,,,,,,,,,,,,,,,,,,
30n20b8.mps.gz,18380.0,576,109706.0,0.451208,8.062840,188.846000,0.390104,18318.0,62,0,...,0.056731,1.000000,0,0,0,0.47,0.01,0.00,0.01,4.21
2club200v15p5scn.mps.gz,200.0,17013,104811.0,0.310556,0.362519,0.500000,-121.222000,200.0,0,0,...,0.004690,0.765000,0,0,0,0.30,0.01,0.00,0.02,9.10
8div-n59k10.mps.gz,6143.0,2065,539151.0,0.242108,9.867550,708.000000,-709.000000,6138.0,5,0,...,0.006379,1.000000,0,0,0,0.52,0.03,0.01,0.03,4.65
22433.mps.gz,429.0,198,3408.0,0.117360,0.200469,0.466942,21240.500000,231.0,0,198,...,0.048224,0.000000,0,0,0,0.00,0.00,0.00,0.00,10.59
10teams.mps.gz,2025.0,230,12150.0,0.020263,0.081960,0.500000,917.000000,1800.0,0,225,...,6.408910,0.086580,0,5,0,0.02,0.00,0.00,0.00,16.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z26.mps.gz,17937.0,850513,1715610.0,0.066037,0.165127,0.500000,-1651.940000,17937.0,0,0,...,0.114323,0.998272,0,0,0,4.49,0.62,0.09,0.65,7.73
8div-n59k12.mps.gz,24575.0,8211,8448020.0,0.251794,19.749700,2844.000000,-2845.000000,24570.0,5,0,...,0.003189,1.000000,0,0,0,37.19,0.42,0.04,0.42,5.89
bley_xl1.mps.gz,5831.0,175620,869391.0,0.056589,0.142278,0.500000,154.390000,5831.0,0,0,...,18.397400,0.000000,0,121,8,154.16,0.15,0.02,0.15,11.55


In [27]:
stats = pd.DataFrame({
    'mean':        df_features.mean(),
    'std':         df_features.std(),
    'range':       df_features.max() - df_features.min(),
    'IQR':         df_features.quantile(0.75) - df_features.quantile(0.25),
    'CV':          df_features.std() / df_features.mean().abs().replace(0, np.nan),
    'skewness':    df_features.apply(skew, nan_policy='omit'),
    'kurtosis':    df_features.apply(lambda x: kurtosis(x, nan_policy='omit')),
    'outlier_%':   df_features.apply(lambda x: ((x < x.mean() - 3*x.std()) | (x > x.mean() + 3*x.std())).mean() * 100)
})

# sorting by biggest disparities.
stats_sorted = stats.sort_values(by='std', ascending=False)
print(stats_sorted)

                                    mean           std         range  \
vcg_var_weight1_avg        -2.105664e+32  5.264160e+33  1.316040e+35   
vcg_var_weight2_avg        -5.763568e+31  1.750076e+33  5.314010e+34   
A_ij_normalized1_avg        3.974767e+31  9.028936e+32  2.050980e+34   
vcg_constr_weight2_avg     -2.292592e+31  6.961331e+32  2.113770e+34   
vcg_constr_weight1_avg     -2.292592e+31  6.961331e+32  2.113770e+34   
...                                  ...           ...           ...   
a_normalized_varcoefs1_avg  6.941019e-02  1.912019e-01  2.585830e+00   
ratio_unbounded_disc        2.815097e-02  1.557916e-01  1.000000e+00   
time_VCG2                   6.492408e-02  1.479268e-01  1.210000e+00   
time_VCG0                   5.056399e-02  1.262054e-01  1.250000e+00   
time_VCG1                   3.088937e-02  9.325034e-02  9.300000e-01   

                                   IQR         CV   skewness    kurtosis  \
vcg_var_weight1_avg           3.972220  25.000000 -24.93996

In [29]:
X = df_features.values
runtimes = df_cplex_runtimes['runtime_sec'].clip(lower=0.005).values
y = np.log10(runtimes)

In [30]:
N_FOLDS   = 10
RANDOM_STATE    = 1234
N_BOOST_ROUNDS = 1_000
EARLY_STOP = 50
VERBOSE_EVAL = True


xgb_params = dict(
    objective      = "reg:squarederror",
    tree_method    = "hist",
    eval_metric    = "rmse",
    random_state   = 1234
)

In [31]:
rmse_scorer = make_scorer(
        lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
        greater_is_better=False
    )

def corr_coeff(y_true, mu):
    """
    Pearson correlation between target and predicted mean.
    Equals Matlab’s ‘cc’.
    """
    return np.corrcoef(y_true, mu)[0, 1]

In [32]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# --- native XGBoost parameter dictionary ---
xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",  # fast histogram builder
    "seed": RANDOM_STATE,
}


def corr_coeff(y_true, y_pred):
    """Pearson correlation coefficient."""
    return np.corrcoef(y_true, y_pred)[0, 1]


kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
rmses = []
ccs = []

for fold, (tr_idx, te_idx) in enumerate(kf.split(X), 1):
    dtrain = xgb.DMatrix(X[tr_idx], label=y[tr_idx])
    dtest = xgb.DMatrix(X[te_idx], label=y[te_idx])

    # Train
    bst = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=N_BOOST_ROUNDS,
        evals=[(dtrain, "train"), (dtest, "valid")],
        early_stopping_rounds=EARLY_STOP,
        verbose_eval=VERBOSE_EVAL,
    )

    # Predict and collect metrics
    preds = bst.predict(dtest, iteration_range=(0, bst.best_iteration + 1))
    rmse = np.sqrt(mean_squared_error(y[te_idx], preds))
    cc = corr_coeff(y[te_idx], preds)

    rmses.append(rmse)
    ccs.append(cc)


print("\n" + "=" * 46)
print(f"{N_FOLDS}-fold RMSE: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
print(f"{N_FOLDS}-fold  CC:  {np.mean(ccs):.3f} ± {np.std(ccs):.3f}")


[0]	train-rmse:1.22637	valid-rmse:1.31801
[1]	train-rmse:1.04335	valid-rmse:1.26678
[2]	train-rmse:0.91378	valid-rmse:1.22993
[3]	train-rmse:0.79807	valid-rmse:1.21319
[4]	train-rmse:0.71273	valid-rmse:1.19459
[5]	train-rmse:0.65038	valid-rmse:1.16847
[6]	train-rmse:0.60085	valid-rmse:1.17774
[7]	train-rmse:0.55819	valid-rmse:1.17769
[8]	train-rmse:0.50611	valid-rmse:1.18202
[9]	train-rmse:0.46095	valid-rmse:1.19214
[10]	train-rmse:0.44270	valid-rmse:1.19561
[11]	train-rmse:0.41175	valid-rmse:1.19409
[12]	train-rmse:0.39425	valid-rmse:1.19591
[13]	train-rmse:0.37144	valid-rmse:1.18837
[14]	train-rmse:0.35399	valid-rmse:1.17880
[15]	train-rmse:0.34461	valid-rmse:1.18363
[16]	train-rmse:0.33006	valid-rmse:1.18269
[17]	train-rmse:0.31381	valid-rmse:1.19095
[18]	train-rmse:0.30431	valid-rmse:1.18858
[19]	train-rmse:0.28646	valid-rmse:1.17066
[20]	train-rmse:0.27430	valid-rmse:1.17301
[21]	train-rmse:0.26591	valid-rmse:1.17062
[22]	train-rmse:0.26085	valid-rmse:1.16930
[23]	train-rmse:0.257

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import xgboost as xgb


def plot_runtime_scatter_xgb(params_dict,
                             X, y_log10,
                             n_folds=10,
                             lower=1e-4, upper=1e4,
                             figsize=(6, 6), random_state=1234):
    """
    Cross‑validated true vs. predicted runtime scatter for *any* XGBoost
    parameter set.  If 'num_boost_round' is absent, it falls back to 1000.

    Parameters
    ----------
    params_dict : dict
        XGBoost parameters; may or may not include 'num_boost_round'.
    X, y_log10  : array‑like
        Features and target in log‑10 seconds.
    """

    # -------------------------- 1. settings -------------------------------
    params     = params_dict.copy()             # do not mutate caller
    num_round  = params.pop("num_boost_round", 1000)

    kf         = KFold(n_splits=n_folds, shuffle=True,
                       random_state=random_state)
    y_pred_log = np.empty_like(y_log10, dtype=float)

    X_np = np.asarray(X)
    y_np = np.asarray(y_log10)

    # -------------------------- 2. CV loop --------------------------------
    for tr_idx, te_idx in kf.split(X_np):
        dtrain  = xgb.DMatrix(X_np[tr_idx], label=y_np[tr_idx])
        booster = xgb.train(params, dtrain,
                            num_boost_round=num_round,
                            verbose_eval=False)

        dtest   = xgb.DMatrix(X_np[te_idx])
        y_pred_log[te_idx] = booster.predict(dtest)

    # -------------------------- 3. seconds & masks ------------------------
    y_true_sec = 10 ** y_np
    y_pred_sec = 10 ** y_pred_log

    inside  = (y_pred_sec >= lower) & (y_pred_sec <= upper)
    outside = ~inside

    # -------------------------- 4. plotting -------------------------------
    fig, ax = plt.subplots(figsize=figsize, dpi=150)
    ax.set_prop_cycle(None)
    ax.set_xscale("log")
    ax.set_yscale("log")

    ax.scatter(y_true_sec[inside],  y_pred_sec[inside],
               s=18, alpha=0.6, edgecolor="k")
    ax.scatter(y_true_sec[outside],
               np.clip(y_pred_sec[outside], lower, upper),
               s=18, alpha=0.6, edgecolor="k", marker="x", color="tab:blue")

    ax.plot([lower, upper], [lower, upper],
            linestyle="--", linewidth=1.0, color="red")

    ax.set_xlim(lower, upper)
    ax.set_ylim(lower, upper)
    tick_vals = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
    tick_lbls = [fr"$10^{{{int(np.log10(t))}}}$" for t in tick_vals]
    ax.set_xticks(tick_vals, tick_lbls)
    ax.set_yticks(tick_vals, tick_lbls)

    ax.set_xlabel("true runtime (s)")
    ax.set_ylabel("predicted runtime (s)")
    ax.set_title(f"XGBoost (No hyp.)")
    ax.grid(which="both", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    return fig, ax

In [ ]:
# `best_params` already contains the keys passed to xgb.train plus
# "num_boost_round" from earlier.
fig, ax = plot_runtime_scatter_xgb(xgb_params, X, y, n_folds=10)
plt.show()

In [ ]:
metrics_summary = {
    "kfold_rmse_mean": float(np.mean(rmses)) if len(rmses) else None,
    "kfold_rmse_std":  float(np.std(rmses))  if len(rmses) else None,
    "kfold_cc_mean":   float(np.mean(ccs))   if len(ccs)   else None,
    "kfold_cc_std":    float(np.std(ccs))    if len(ccs)   else None,
    "kfold_rmse":      [float(v) for v in rmses] if len(rmses) else [],
    "kfold_cc":        [float(v) for v in ccs]   if len(ccs)   else [],
}
save_metrics("kfold_summary", metrics_summary)

feature_names = list(df_features.columns)
dtrain_full = xgb.DMatrix(X, label=y, feature_names=feature_names)

cv_table = xgb.cv(
    params=xgb_params,
    dtrain=dtrain_full,
    num_boost_round=N_BOOST_ROUNDS,
    nfold=N_FOLDS,
    early_stopping_rounds=EARLY_STOP,
    metrics="rmse",
    seed=RANDOM_STATE,
    verbose_eval=False,
)

best_num_boost_round = len(cv_table)
best_row = cv_table.iloc[-1]
save_metrics("xgb_cv", {
    "best_num_boost_round": int(best_num_boost_round),
    "train_rmse_mean": float(best_row["train-rmse-mean"]),
    "train_rmse_std":  float(best_row["train-rmse-std"]),
    "test_rmse_mean":  float(best_row["test-rmse-mean"]),
    "test_rmse_std":   float(best_row["test-rmse-std"]),
})
cv_table.to_csv(RUN_DIR / "metrics" / "xgb_cv_results.csv", index=True)
print(f"Best num_boost_round from CV: {best_num_boost_round}")

final_model = xgb.train(
    params=xgb_params,
    dtrain=dtrain_full,
    num_boost_round=best_num_boost_round,
    verbose_eval=False,
)

save_model(final_model, name="xgb-runtime-miplib-default-final")